In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import json
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import RandomContrast
import numpy as np
from PIL import Image

from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
# Please reset the paths here to the training and test datatrain_path = '../input/currency-prediction-challenge/data/train' # train data
test_path = '../input/currency-prediction-challenge/data/test'  # test data
train_labels = pd.read_csv('../input/currency-prediction-challenge/data/train.csv') # training labels
test_labels = pd.read_csv('../input/currency-prediction-challenge/data/test.csv')  # testing labels
label_val=json.load(open('../input/currency-prediction-challenge/data/cat_to_name.json'))
file=[]
label=[]
for ind in train_labels.index:
    label.append(int(list(label_val.keys())[list(label_val.values()).index(train_labels['value'][ind]+','+train_labels['currency'][ind]+','+train_labels['country'][ind])]))
for fil in sorted(os.listdir(train_path),key=lambda x: int(x.split(".")[0]) ):
    img=Image.open(train_path+'/'+fil).resize((128,128))
    img_dat1= np.array(img)
    file.append(img_dat1/255)

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(file,label,test_size=0.20,random_state=5)
train=np.array(X_train)
train_label=np.array(y_train,dtype = 'float32')
del file
del label

In [ ]:
val=np.array(X_val)
val_label=np.array(y_val,dtype='float')
del X_val
del y_val

In [ ]:
from keras.layers import GaussianNoise
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(128,128,3),
                   pooling='avg',classes=212,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=True
resnet_model = keras.Sequential()
resnet_model.add(layers.RandomContrast(0.2))
resnet_model.add(pretrained_model)
resnet_model.add(layers.Flatten())
resnet_model.add(GaussianNoise(0.18))
resnet_model.add(layers.BatchNormalization())
resnet_model.add(layers.Dense(280, activation='relu',kernel_regularizer=keras.regularizers.L1(7e-4)))
resnet_model.add(layers.Dense(212, activation='softmax',kernel_regularizer=keras.regularizers.L1(8e-5)))

In [ ]:
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4.5e-4),
              loss = 'sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy']
              )
resnet_model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./Epochdat',
    save_weights_only=True,
    monitor='val_sparse_categorical_accuracy',
    mode='max',
    save_best_only=True)
resnet_model.fit(train,train_label,batch_size=64, epochs=60, verbose=1,validation_data=(val,val_label),callbacks=[resnet_model_checkpoint_callback])

In [ ]:
resnet_model.load_weights('./Epochdat')
pred=[]
name=[]
pred_label=[]
for f in sorted(os.listdir('../input/currency-prediction-challenge/data/test'),key=lambda x: int(x.split(".")[0]) ):
    name.append(os.path.basename(f))
    img=Image.open('../input/currency-prediction-challenge/data/test'+'/'+f).resize((128,128))
    iml=np.array(img)/255
    pred.append(iml)
    prel=np.array(pred)
    pred_label.append(np.argmax(resnet_model.predict(prel)))
    pred.clear()

In [ ]:
submissions=pd.DataFrame({'label':pred_label},index=name)
submissions = submissions.rename_axis('name')
submissions.to_csv('submission.csv')
submissions